In [ ]:
#Preparando o script para que o algoritmo seja processado na placa do video do PC
import tensorflow as tf
from keras.models import Sequential  # Certifique-se de importar Sequential

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        #Alocar memória de vídeo de forma dinâmica
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

# Agora defina o modelo
with tf.device('/GPU:0'):
    model = Sequential()

############################################### PREVISÃO PARA VALORES DIARIOS ###############################################################################################

#Preparando o algoritmo RNN para previsões da Eto diaria  
    
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from tensorflow.keras.layers import Dense
from keras.layers import SimpleRNN
from sklearn.metrics import mean_squared_error
from math import sqrt
from keras.layers import Dropout

#Importar o banco de dados 
dados = pd.read_csv('caminho_do_arquivo.csv', sep=';')  

print(dados.head())
print(dados.columns)

#Normaliza as features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(dados[['rad', 'tmed', 'umi', 'vento', 'ETo']])

#Converte para DataFrame
scaled_df = pd.DataFrame(scaled_data, columns=['rad', 'tmed', 'umi', 'vento', 'ETo'])

#Transforma a série temporal em um formato supervisionado
def series_to_supervised(data, n_in=24, n_out=1):
    cols, names = list(), list()
    #input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(data.shift(i))
        names += [f'{col_name}(t-{i})' for col_name in data.columns]

    #forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(data.shift(-i))
        if i == 0:
            names += [f'{col_name}(t)' for col_name in data.columns]
        else:
            names += [f'{col_name}(t+{i})' for col_name in data.columns]

    agg = pd.concat(cols, axis=1)
    agg.columns = names
    agg.dropna(inplace=True)
    return agg

reframed = series_to_supervised(scaled_df, n_in=24, n_out=1)

#Divide os dados em treino e teste
values = reframed.values
n_train = int(0.9 * len(reframed))  #Valor ajustavel
train = values[:n_train, :]
test = values[n_train:, :]

#Verifica a quantidade de dados nos conjuntos de treino e teste
print(f"Dimensões do conjunto de treinamento: {train.shape}")
print(f"Dimensões do conjunto de teste: {test.shape}")

#Separa inputs e outputs
train_X, train_y = train[:, :-1], train[:, -5]  #Seleciona a coluna ETo
test_X, test_y = test[:, :-1], test[:, -5]

#Redimensiona o input para o formato 3D [amostras, passos de tempo, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))


#Definindo o modelo RNN com camadas Dropout
from keras import regularizers

model = Sequential()
model.add(SimpleRNN(100, input_shape=(train_X.shape[1], train_X.shape[2]), return_sequences=True))
model.add(SimpleRNN(50, return_sequences=True))
model.add(Dense(1))

#Compilando o modelo
model.compile(loss='mae', optimizer='adam')


#Adicionando a função early stopping para evitar o overfitting
from keras.callbacks import EarlyStopping


early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

#Treinando o modelo com EarlyStopping
history = model.fit(train_X, train_y, epochs=1000, batch_size=90, 
                    validation_data=(test_X, test_y), verbose=2, 
                    shuffle=False, callbacks=[early_stopping])



import matplotlib.pyplot as plt

#Plota o histórico de treinamento e validação
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='validation')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt

#Faz as previsões com o conjunto de teste
y_pred = model.predict(test_X)

#Caso a saída do modelo seja tridimensional, redimensione-a para bidimensional
if y_pred.ndim > 2:
    y_pred = y_pred.squeeze()


#Calcula o RMSE (Root Mean Squared Error)
rmse = sqrt(mean_squared_error(test_y, y_pred))
print('RMSE:', rmse)

#Calcular o MSE (Mean Squared Error)
mse = mean_squared_error(test_y, y_pred)
print('MSE:', mse)

#Calcula o MAE (Mean Absolute Error)
mae = mean_absolute_error(test_y, y_pred)
print('MAE:', mae)


#Fazendo a previsão
future_input = test_X[-1].reshape((1, 1, test_X.shape[2]))  #Usa o último conjunto de teste como entrada
#Faz a revisão para os próximos 15 dias 
future_predictions = []

for i in range(15):
    next_prediction = model.predict(future_input)
    future_predictions.append(next_prediction[0, 0])
    future_input = np.roll(future_input, -1, axis=1)
    future_input[0, 0, -1] = next_prediction[0, 0]

#Inverte a escala das previsões
scaled_inv = scaler.inverse_transform(np.concatenate((np.zeros((15, 4)), np.array(future_predictions).reshape(-1, 1)), axis=1))
predicted_ETo = scaled_inv[:, -1]

#Exibe o resultado das previsões
print('Previsões para os próximos 15 dias:')
print(predicted_ETo)

#Salva as previsões em uma planilha

df_predictions = pd.DataFrame({'Predicted_ETo': predicted_ETo})
df_predictions.to_excel('RNN_diario.xlsx', index=False)


############################################### PREVISÃO PARA VALORES HORARIOS ###############################################################################################

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt

#Importando o banco de dados 
dados = pd.read_csv('caminho_do_arquivo.csv', sep=';')  

print(dados.head())
print(dados.columns)

#Normalizando as features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(dados[['rad', 'tmed', 'umi', 'vento', 'ETo']])

#Convertendo para DataFrame
scaled_df = pd.DataFrame(scaled_data, columns=['rad', 'tmed', 'umi', 'vento', 'ETo'])

#Transformando a série temporal em um formato supervisionado
def series_to_supervised(data, n_in=1, n_out=1):
    cols, names = list(), list()
    for i in range(n_in, 0, -1):
        cols.append(data.shift(i))
        names += [f'{col_name}(t-{i})' for col_name in data.columns]

    for i in range(0, n_out):
        cols.append(data.shift(-i))
        if i == 0:
            names += [f'{col_name}(t)' for col_name in data.columns]
        else:
            names += [f'{col_name}(t+{i})' for col_name in data.columns]

    agg = pd.concat(cols, axis=1)
    agg.columns = names
    agg.dropna(inplace=True)
    return agg

reframed = series_to_supervised(scaled_df, n_in=24, n_out=1)

#Dividindo o conjunto em treino e teste
values = reframed.values
n_train = int(0.7 * len(reframed))  #Valor ajustavel 
train = values[:n_train, :]
test = values[n_train:, :]

#Separando inputs e outputs
train_X, train_y = train[:, :-1], train[:, -1]
test_X, test_y = test[:, :-1], test[:, -1]

#Redimensionando o input para 3D [amostras, passos de tempo, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))

#Definindo a arquitetura da rede RNN
model = Sequential()
model.add(SimpleRNN(100, input_shape=(train_X.shape[1], train_X.shape[2]), return_sequences=True))
model.add(SimpleRNN(50, return_sequences=True))
model.add(Dense(1))

#Compilando o modelo
model.compile(loss='mae', optimizer='adam')

#Adicionando a função early stopping para evitar o overfitting
from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

#Treinando o modelo com EarlyStopping
model.fit(train_X, train_y, epochs=100, batch_size=90, 
          validation_data=(test_X, test_y), verbose=2, 
          shuffle=False, callbacks=[early_stopping])


#Calculando as métricas de erro

#Calcula o RMSE (Root Mean Squared Error)
rmse = sqrt(mean_squared_error(test_y, y_pred))
print('RMSE:', rmse)

#Calcula o MSE (Mean Squared Error)
mse = mean_squared_error(test_y, y_pred)
print('MSE:', mse)

#Calcula o MAE (Mean Absolute Error)
mae = mean_absolute_error(test_y, y_pred)
print('MAE:', mae)


#Fazendo as previsões
#Número de horas por dia
hours_per_day = 24

#Número de dias futuros para prever
num_days = 15

#Definindo a sequência inicial para fazer as previsões
last_test_sequence = test_X[-1].reshape((1, test_X.shape[1], test_X.shape[2]))

#Lista para armazenar as previsões
future_predictions = []

for day in range(num_days):
    for hour in range(hours_per_day):
        next_prediction = model.predict(last_test_sequence)
        future_predictions.append(next_prediction[0, 0]) 
        last_test_sequence = np.roll(last_test_sequence, -1, axis=1)
        last_test_sequence[:, -1, -1] = next_prediction

#Exibe as previsões para os próximos 15 dias (cada dia contendo 24 horas)
print("Previsões para os próximos 15 dias (24 horas por dia):")
print(future_predictions)



import pandas as pd
import numpy as np

#Cria um DataFrame para armazenar os valores previstos
data = np.array(future_predictions).reshape(num_days, hours_per_day)
df = pd.DataFrame(data)

#Adiciona rótulos de coluna (horas do dia)
hours_labels = [f'{hour}:00' for hour in range(hours_per_day)]
df.columns = hours_labels

#Adiciona rótulos de linha (dias)
days_labels = [f'Dia {day + 1}' for day in range(num_days)]
df.index = days_labels

#Exporta o DataFrame para um arquivo Excel
nome_arquivo = 'RNN_horario.xlsx'  
df.to_excel(nome_arquivo)

